In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

from sklearn import metrics
import numpy as np
import pandas as pd
import random
import math


import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import timm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,random_split

continued the SETI-test1.

I thought it would more nice modify residual layer to auto-encoder layer.

pair (0,1), (2,3), (4,5) 2 images as one set in Encoder.

In [ ]:
def dataframe(df, istrain=True):
    if istrain:
        mode = 'train'
    else:
        mode = 'test'
    df['filepath'] = df.id.apply(lambda x: '../input/seti-breakthrough-listen/'+ f'{mode}/'+str(x[0])+f'/{x}.npy')
    return df
test_df = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
test_df = dataframe(test_df, istrain=False)
test_df['target'] = 0
train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df = dataframe(train_df)
train_df.head(2)

In [ ]:
class setiDataset(Dataset):
    def __init__(self, df, transform = None): 
        self.df = df
        self.transform = transform
    
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df.target.iloc[idx]
    
    def loadfile(self, filepath):
        image = np.load(filepath).astype('float32')
        return image
    
    def __getitem__(self, idx):
        filepath, target = self.fileinfo(idx)
        image = self.loadfile(filepath)
        return  torch.tensor(image, dtype=torch.float), torch.tensor(target, dtype=torch.long)
    
    def __len__(self):
        return len(self.df)

In [ ]:
# Encoder
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding = 1 if kernel_size == 3 else 0, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.vals = nn.Parameter(torch.zeros(out_channels)) 
        nn.init.kaiming_normal_(self.conv.weight, mode='fan_in', nonlinearity='leaky_relu')
        
    def forward(self, x):
        x = self.conv(x)
        x = F.leaky_relu(x, 0.1)
        x = self.bn(x)
        x += self.vals.view(1, self.bn.num_features, 1, 1).expand_as(x)
        x = x * torch.tanh(F.softplus(x))
        return x

class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, out_channels, 3)
        self.conv2 = ConvBlock(out_channels, out_channels, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x * torch.tanh(F.softplus(x))
        return x
    
class Encoder(nn.Module):
    def __init__(self, in_channels = 2, res_level = 4):
        super(Encoder,self).__init__()
        self.res_layer = nn.Sequential(
            *[ResBlock(in_channels, in_channels) for _ in range(res_level)]
        )
    
    def forward(self, x):
        x1 = x[:,:2,...]
        x2 = x[:,2:4,...]
        x3 = x[:,4:,...]
        
        x1 = self.res_layer(x1)
        x2 = self.res_layer(x2)
        x3 = self.res_layer(x3)
        #maybe it would be effective using channelwise 
        
        x = torch.cat([x1,x2,x3], dim = 1)
        return x    

In [ ]:
class effModel(nn.Module):
    def __init__(self, in_chans = 6, num_classes = 1, pretrained=True):
        super(effModel,self).__init__()
        
        self.model = timm.create_model('efficientnet_b0', 
                                       in_chans=in_chans, 
                                       num_classes=num_classes,
                                       pretrained=pretrained)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def train(data_loader, encoder, model, optimizer, device):
    model.train()
    encoder.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        images, targets = data
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        def closure():
            optimizer.zero_grad()
            encode = encoder(images)
            output = model(encode)
            loss = nn.BCEWithLogitsLoss()(output, targets.view(-1,1))
            loss += 0.15*nn.L1Loss()(encode,images)
            loss.backward()
            return loss
        optimizer.step(closure)

In [ ]:
def evaluate(data_loader, encoder, model, device):
    model.eval()
    encoder.eval()
    
    final_targets = []
    final_outputs = []
    validate_losses = 0
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = encoder(images)
            output = model(output)
            loss = nn.BCEWithLogitsLoss()(output, targets.view(-1, 1))
            validate_losses += loss.item()


            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
    return final_outputs, final_targets, validate_losses/len(data_loader)

In [ ]:
def submission(data_loader, encoder, model, device):
    model.eval()
    encoder.eval()
    
    final_outputs = []
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            
            output = encoder(images)
            output = model(output)
            output = output.detach().cpu().numpy().tolist()
            final_outputs.extend(output)
    return final_outputs

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device('cpu')
    print("GPU not available, CPU used")

In [ ]:
Batch_Size = 16
init_dataset = setiDataset(train_df)
lengths = [int(len(init_dataset)*0.8), int(len(init_dataset)*0.2)]

train_dataset, valid_dataset = random_split(init_dataset, lengths)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size, shuffle=False)

encoder = Encoder()
encoder.to(device)
model = effModel()
model.to(device)

params = list(model.parameters()) + list(encoder.parameters())
optimizer=  torch.optim.Adam(params, lr=1e-4, eps=1e-5)

for epoch in range(10):
    print(epoch)
    train(train_loader, encoder, model, optimizer, device)
    valid_pred, target, valid_loss = evaluate(valid_loader, encoder, model, device)
    roc_auc = metrics.roc_auc_score(target, valid_pred)
    print(f"Epoch={epoch}, Valid Loss={valid_loss}, Valid ROC AUC={roc_auc}")

In [ ]:
image, target = setiDataset(train_df[train_df.target==1].reset_index(drop=True))[1028]

plt.figure(figsize=(16,8))
plt.subplot(2,3,1)
plt.imshow(image[0], aspect='auto')
plt.subplot(2,3,2)
plt.imshow(image[2], aspect='auto')
plt.subplot(2,3,3)
plt.imshow(image[4], aspect='auto')
plt.show()
print('original, target:1')

image = image.unsqueeze(dim=0).to(device)
image = encoder(image)
image = image.to('cpu').view(6,273,256)
image = image.detach().numpy()
plt.figure(figsize=(16,8))
plt.subplot(2,3,1)
plt.imshow(image[0], aspect='auto')
plt.subplot(2,3,2)
plt.imshow(image[2], aspect='auto')
plt.subplot(2,3,3)
plt.imshow(image[4], aspect='auto')
plt.show()
print('encoder, target:1')

image, target = setiDataset(train_df[train_df.target==1].reset_index(drop=True))[512]

plt.figure(figsize=(16,8))
plt.subplot(2,3,1)
plt.imshow(image[0], aspect='auto')
plt.subplot(2,3,2)
plt.imshow(image[2], aspect='auto')
plt.subplot(2,3,3)
plt.imshow(image[4], aspect='auto')
plt.show()
print('original, target:1')

image = image.unsqueeze(dim=0).to(device)
image = encoder(image)
image = image.to('cpu').view(6,273,256)
image = image.detach().numpy()
plt.figure(figsize=(16,8))
plt.subplot(2,3,1)
plt.imshow(image[0], aspect='auto')
plt.subplot(2,3,2)
plt.imshow(image[2], aspect='auto')
plt.subplot(2,3,3)
plt.imshow(image[4], aspect='auto')
plt.show()
print('encoder, target:1')

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

test_dataset = setiDataset(test_df)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, shuffle=False)
pred = submission(test_loader, encoder, model, device)
sub = test_df[['id']].copy()
sub['target'] = sigmoid(np.array(pred).flatten())
sub.to_csv('submission.csv', index=None)
